# Martian SDK Quickstart Guide

In [2]:
# Imports

import openai
from openai.types.chat import (
    chat_completion,
    chat_completion_message,
)

from martian_apart_hack_sdk import judge_specs, martian_client, utils

## Load Credentials
You must have a .env file with the following values set:

1. `MARTIAN_API_URL` - withmartian.com/api
1. `MARTIAN_ORG_ID` - your Martain organization ID
1. `MARTIAN_API_KEY` - your personal API key

In [3]:
# Load the config and make a client.
config = utils.load_config()
client = martian_client.MartianClient(
    api_url=config.api_url,
    api_key=config.api_key,
    org_id=config.org_id,
)

## Martian Gateway

You can use Martian as a gateway to access a number of different LLM providers.
To do so, you start by making an OpenAI client with the base_url set to the Martian API URL + "/openai/v2".
Then you can use the client as you would when working with OpenAI.

The list of available models are:

- gpt-4.5-preview
- gpt-4.1
- gpt-4.1-mini
- gpt-4.1-nano
- gpt-4o
- gpt-4o-mini
- o3-mini

- claude-3-opus-latest
- claude-3-5-haiku-latest
- claude-3-5-sonnet-latest
- claude-3-7-sonnet-latest

- together/deepseek-ai/DeepSeek-R1
- together/deepseek-ai/DeepSeek-V3
- together/mistralai/Mistral-Small-24B-Instruct-2501
- together/nvidia/Llama-3.1-Nemotron-70B-Instruct-HF
- together/meta-llama/Llama-3.3-70B-Instruct-Turbo
- together/meta-llama/Meta-Llama-3.1-405B-Instruct-Turbo
- together/Qwen/Qwen2.5-72B-Instruct-Turbo
- together/Qwen/Qwen2.5-Coder-32B-Instruct
- together/google/gemma-2-27b-it

In [36]:
# Create the client.
openai_client = openai.OpenAI(
    api_key=config.api_key,
    base_url=config.api_url + "/openai/v2"
)

# Create a request.
gpt_nano_chat_completion_response = openai_client.chat.completions.create(
    model="gpt-4.1-nano",
    messages=[{"role": "user", "content": "What is the capital of France?"}],
)
claude_3_haiku_chat_completion_response = openai_client.chat.completions.create(
    model="claude-3-5-haiku-latest",
    messages=[{"role": "user", "content": "What is the capital of France?"}],
)
gemma_2_chat_completion_response = openai_client.chat.completions.create(
    model="gemma-2-27b-it",
    messages=[{"role": "user", "content": "What is the capital of France?"}],
)

# Get the response.
print("gpt-4.1-nano says:", gpt_nano_chat_completion_response.llm_response["choices"][0]["message"]["content"])  # type: ignore
print("claude-3-5-haiku-latest says:", claude_3_haiku_chat_completion_response.llm_response["choices"][0]["message"]["content"])  # type: ignore
print("gemma-2-27b-it says:", gemma_2_chat_completion_response.llm_response["choices"][0]["message"]["content"])  # type: ignore

GPT-4o-mini says: The capital of France is **Paris**. 🇫🇷 



## Judging

In [3]:
# Create a JudgeSpec

rubric = """
You are tasked with evaluating whether a restaurant recommendation is good.
The scoring is as follows:
- 1: If the recommendation doesn't meet any of the criteria.
- 2: If the recommendation meets only some small part of the criteria.
- 3: If the recommendation is reasonable, but not perfect.
- 4: If the recommendation is almost perfect.
- 5: If the recommendation is perfect.
""".strip()

rubric_judge_spec = judge_specs.RubricJudgeSpec(
    model_type="rubric_judge",
    rubric=rubric,
    # TODO: Clearly communicate which models are available.
    model="openai/openai/gpt-4o",
    min_score=1,
    max_score=5,
)

In [4]:
# Run the judge spec.
# TODO: Go through advanced judge building.

chat_request_text = "What is a good Chinese restaurant in downtown San Francisco?"
chat_response_text = "I couldn't find a good Mexican restaurant near you."

completion_request = {
    "model": "openai/openai/gpt-4o-mini",
    "messages": [{"role": "user", "content": chat_request_text}],
}
chat_completion_response = chat_completion.ChatCompletion(
    id="123",
    choices=[
        chat_completion.Choice(
            finish_reason="stop",
            index=0,
            message=chat_completion_message.ChatCompletionMessage(
                role="assistant",
                content=chat_response_text,
            ),
        )
    ],
    created=0,
    model="gpt-4o",
    object="chat.completion",
    service_tier=None,
)

evaluation_result = client.judges.evaluate_judge_spec(
    rubric_judge_spec.to_dict(),
    completion_request=completion_request,
    completion_response=chat_completion_response,
)

print(f"User: {chat_request_text}")
print(f"Assistant: {chat_response_text}")
print(f"Evaluation result: {evaluation_result}")

User: What is a good Chinese restaurant in downtown San Francisco?
Assistant: I couldn't find a good Mexican restaurant near you.
Evaluation result: JudgeEvaluation(score=1, reason="\nThe assistant's response does not address the user's specific request for a Chinese restaurant in downtown San Francisco. Instead, it incorrectly discusses Mexican restaurants, which is unrelated to the user's question. According to the rubric, the recommendation does not meet any of the criteria required, as it completely fails to provide a relevant response. Thus, it deserves the lowest score possible.\n", cost=0.0015225)


In [5]:
# Once you are happy with the judge, you can save it.

judge_id = "restaurant-recommendation-reviewer"

judge = client.judges.create_judge(
    judge_id=judge_id,
    judge_spec=rubric_judge_spec,
    description="A judge that rates how good restaurant recommendations are."
)

print(f"Created a judge: {judge}")

Created a judge: Judge(id='restaurant-recommendation-reviewer', version=1, description='A judge that rates how good restaurant recommendations are.', createTime='2025-05-23T15:52:53.326120Z', name='organizations/68275a4af88f39d6440c1050/judges/restaurant-recommendation-reviewer', judgeSpec={'extract_judgement': {'extraction_fields': [{'extraction_pattern': '<rationale>(.*?)</rationale>', 'field_type': 'STRING', 'match_index': -1, 'name': 'rationale', 'required': True}, {'extraction_pattern': '<score>(.*?)</score>', 'field_type': 'FLOAT', 'match_index': -1, 'name': 'score', 'required': True}], 'model_type': 'regex_extractor'}, 'extract_variables': {'extraction_fields': [{'extraction_pattern': '', 'field_type': 'STRING', 'match_index': 0, 'name': 'content', 'required': True}], 'model_type': 'default_extractor'}, 'max_score': 5, 'min_score': 1, 'model': 'openai/openai/gpt-4o', 'model_type': 'rubric_judge', 'postscript': "Here's the conversation you are judging:\n<content>\n${content}\n</c

In [6]:
# You can then call the judge:

evaluation_result = client.judges.evaluate_judge(
    judge,
    completion_request=completion_request,
    completion_response=chat_completion_response,
)

print(f"Judge response: {evaluation_result}")

{'judgement': {'score': 1, 'reason': "\nThe assistant's response is entirely irrelevant to the user's question. The user asked for a recommendation of a Chinese restaurant in downtown San Francisco, but the assistant responded with information about a Mexican restaurant, which is not what was requested. This means that the response does not meet any of the criteria set out for evaluating a good restaurant recommendation. Therefore, based on the rubric, the assistant's response merits the lowest possible score because it fails to provide any helpful information regarding the user's request.\n", 'cost': 0.0017625}}
Judge response: JudgeEvaluation(score=1, reason="\nThe assistant's response is entirely irrelevant to the user's question. The user asked for a recommendation of a Chinese restaurant in downtown San Francisco, but the assistant responded with information about a Mexican restaurant, which is not what was requested. This means that the response does not meet any of the criteria 

In [7]:
# Once you have created some judges, you may want to list them all:

all_judges = client.judges.list()
print("Judges:")
print(*[f"\t- {j}" for j in all_judges])

# Or get a specific judge:

retrieved_judge = client.judges.get(
    judge_id="restaurant-recommendation-reviewer",
    version=1,
)
print(f"\nRetrieved judge: {retrieved_judge}")

# Or update the judge, creating a new version:


new_rubric = """
You are tasked with evaluating whether a restaurant recommendation is good.
The scoring is as follows:
- 1: If the recommendation doesn't meet any of the criteria.
- 2: If the recommendation meets only some small part of the criteria.
- 3: If the recommendation is reasonable, but not perfect.
- 4: If the recommendation is almost perfect.
- 5: If the recommendation is perfect.
""".strip()

new_rubric_judge_spec = judge_specs.RubricJudgeSpec(
    model_type="rubric_judge",
    rubric=rubric,
    # TODO: Clearly communicate which models are available.
    model="openai/openai/gpt-4o",
    min_score=1,
    max_score=5,
)

new_judge_spec = client.judges.update_judge(
    judge_id="restaurant-recommendation-reviewer",
    judge_spec=new_rubric_judge_spec,
)

print(f"\nNew judge spec: {new_judge_spec}")

Judges:
	- Judge(id='restaurant-recommendation-reviewer', version=1, description='A judge that rates how good restaurant recommendations are.', createTime='2025-05-23T15:52:53.326120Z', name='organizations/68275a4af88f39d6440c1050/judges/restaurant-recommendation-reviewer', judgeSpec=None) 	- Judge(id='my-rubric-judge', version=11, description='', createTime='2025-05-23T15:40:09.091441Z', name='organizations/68275a4af88f39d6440c1050/judges/my-rubric-judge', judgeSpec=None) 	- Judge(id='rubric-judge', version=1, description='', createTime='2025-05-23T15:22:05.194690Z', name='organizations/68275a4af88f39d6440c1050/judges/rubric-judge', judgeSpec=None) 	- Judge(id='new-quality-rubric-judge', version=2, description='A judge that evaluates response quality based on accuracy, completeness, clarity, and helpfulness', createTime='2025-05-22T14:01:07.021971Z', name='organizations/68275a4af88f39d6440c1050/judges/new-quality-rubric-judge', judgeSpec=None) 	- Judge(id='rubric-judge-test-id', versi

## Routing